# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [4]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn.yml
    !mamba env update -n base -f environment_amn.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['README.md', 'Duplicate_Model.ipynb', 'Build_Model_Dense.ipynb', 'Build_Dataset.py', 'Dataset_experimental', '.ipynb_checkpoints', '.git', 'Build_Experimental.ipynb', 'Reservoir', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'Duplicate_Model.py', 'LICENSE', 'Build_Dataset.ipynb', 'Dataset_input', '__pycache__', 'Build_Experimental.py', 'old', 'environment_amn.yml', 'Build_Model.py', 'Build_Model.ipynb', '.DS_Store']


## (3) Hybrid model: examples of trainable AMN model (neural and mechanistic) with FBA simulated training sets or experimental datasets

### AMN LP or QP Neural + Mechanistic programs

In [2]:
# Create, train and evaluate AMN_QP o models with FBA simulated training set for E. coli core
# with EB or UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_QP', 
                     timestep = timestep, learn_rate=0.01,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs=500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
    

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.97 loss-train = 0.002241 loss-test = 0.002233 iter=0
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.003301 loss-test = 0.003456 iter=0
nbr parameters: 8904
6/6 [==============================] - 0s 2ms/step
train = 0.96 test = 0.96 loss-tra

In [3]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for E. coli core
# with UB with a mechanistic layer

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_LP', 
                     timestep = timestep, learn_rate=1.0e-6,
                     scaler=True,
                     n_hidden = 1, hidden_dim = 50,
                     epochs= 500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
# LP keeps track of boundary fluxes in b_int and b_ext
# and these are different in EB or UB modes
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:] if model.mediumbound == 'UB' else model.b_int[ID,:]
bint  = model.b_int if model.mediumbound == 'UB' else model.b_ext
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0) if model.mediumbound == 'UB' \
else np.delete(model.b_int, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
reservoir.save(reservoirfile)
reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    reservoir.b_ext, reservoir.b_int =  btest, bint
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
    

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.96 test = 0.96 loss-train = 0.002815 loss-test = 0.002802 iter=0
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.98 loss-train = 0.002668 loss-test = 0.002781 iter=0
nbr parameters: 25152
6/6 [==============================] - 0s 2ms/step
train = 0.97 test = 0.97 loss-train = 0.002129 loss-test = 0.002127 iter=0
nbr pa

In [10]:
# Create, train and evaluate AMN_QP models with FBA simulated training set for iML1515
# with EB or UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.01,
              n_hidden = 1, hidden_dim = 500,
              epochs = 25, xfold = 5, 
              verbose=True) 
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    reservoir.model.b_ext = btest
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)
 

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_QP
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
gradient learn rate: 0.01
gradient decay rate: 0.9
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
QP input shape (9900, 38) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 500 550 relu True
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (None, 1) (None, 1) (None, 1) (None, 1) (None, 550) (None, 1104)
Model: "model_95"
_____________________

In [11]:
# Create, train and evaluate AMN_LP models with FBA simulated training set for iML1515
# with UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=1.0e-6,
              n_hidden = 1, hidden_dim = 250,
              epochs = 25, xfold = 5, 
              verbose=True) 

ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
btest = model.b_ext[ID,:]
bint = model.b_int
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
btrain = np.delete(model.b_ext, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y, model.b_ext, model.b_int = Xtrain, Ytrain, btrain, bint
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    reservoir.b_ext, reservoir.b_int =  btest, bint
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)


number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_LP
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 250
activation function: relu
training epochs: 25
training regression: True
training learn rate: 0.001
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
LP input shape (9900, 3235) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 550 relu True
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 250 2716 linear True
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (None, 1) (None, 1) (None, 1) (None, 1) (None, 550) (None, 11

### AMN Wt program

In [9]:
# Create, train and evaluate AMN_Wt models with FBA simulated training set for E. coli core
# with UB (not working with M1 chips)

# What you can change 
seed = 10
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'e_coli_core_UB' # can change EB by UB
timestep = 4
# End of What you can change
  
# Create model 90% for training 10% for testing
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
                     objective=['BIOMASS_Ecoli_core_w_GAM'],  
                     model_type='AMN_Wt', 
                     timestep = timestep,
                     n_hidden = 1, hidden_dim = 50,
                     scaler=True,
                     train_rate=1e-2,
                     epochs=500, xfold=5,
                     verbose=True)
ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0)
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=False)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir, verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)

number of reactions:  154 154
number of metabolites:  72
filtered measurements size:  1
training file: ./Dataset_model/e_coli_core_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 20
model output dim: 1
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
training epochs: 500
training regression: True
training learn rate: 0.01
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000919 loss-test = 0.000912 iter=0
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000933 loss-test = 0.000990 iter=0
nbr parameters: 13262
6/6 [==============================] - 0s 2ms/step
train = 0.99 test = 0.99 loss-train = 0.000908 loss-test = 0.000904 iter=0
nbr par

In [12]:
# Create, train and evaluate AMN_QP models with FBA simulated training set for iML1515
# with EB or UB with a mechanistic layer 
# This cell takes several hours to execute

# What you can change 
seed = 1
ratio_test = 0.1 # part of the training set removed for test
np.random.seed(seed=seed)  
trainname = 'iML1515_UB'
timestep = 4
# End of What you can change
  
# Create model 
trainingfile = DIRECTORY+'Dataset_model/'+trainname
model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_Wt',
              scaler = True,
              timestep = timestep, 
              n_hidden = 1, hidden_dim = 500,
              train_rate=1e-2,
              epochs = 100, xfold = 5, 
              verbose=True) 

ID = np.random.choice(model.X.shape[0], 
                      size=int(model.X.shape[0]*ratio_test), 
                      replace=False)
Xtest,  Ytest  = model.X[ID,:], model.Y[ID,:]
Xtrain, Ytrain = np.delete(model.X, ID, axis=0), np.delete(model.Y, ID, axis=0) 
model.printout()

# Train and evaluate
reservoirname = trainname+'_'+model.model_type
reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
start_time = time.time()
model.X, model.Y = Xtrain, Ytrain
reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
delta_time = time.time() - start_time

# Printing cross-validation results
printout(reservoirname, stats, model, delta_time)

# Save, reload and run idependent test set
#reservoir.save(reservoirfile)
#reservoir.load(reservoirfile)
reservoir.printout()
if len(Xtest) > 0:
    start_time = time.time()
    reservoir.X, reservoir.Y = Xtest, Ytest
    X, Y = model_input(reservoir,verbose=False)
    pred, stats = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    delta_time = time.time() - start_time
    printout('Test set', stats, model, delta_time)

number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
training file: ./Dataset_model/iML1515_UB
model type: AMN_Wt
model scaler: 1.0
model input dim: 38
model output dim: 1
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 500
activation function: relu
training epochs: 100
training regression: True
training learn rate: 0.01
training dropout: 0.25
training batch size: 5
training validation iter: 0
training xfold: 5
training early stopping: False
AMN scaler 11.0
Wt input shape (9900, 4, 38) (9900, 4)
-------train [   0    1    3 ... 9897 9898 9899]
-------test  [   2    4    6 ... 9883 9891 9893]
number of reactions:  550 550
number of metabolites:  1083
filtered measurements size:  1
Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to             

### AMN -QP, -LP or -Wt with experimental training set

In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


In [ ]:
# Create, train and evaluate AMN_QP models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_LP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')


In [ ]:
# Create, train and evaluate AMN_Wt models on experimental training set with UB 
# Repeat the process with different seeds
# This cell takes several hours to execute

Maxloop, Q2, PRED = 3, [], []

for Nloop in range(Maxloop):
    # What you can change 
    seed = Nloop+1
    np.random.seed(seed=seed)  
    trainname = 'iML1515_EXP_UB' 
    timestep = 4
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'], 
              model_type = 'AMN_Wt',
              scaler = True,
              timestep = timestep,
              n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              epochs = 1000, xfold = 10, 
              verbose=True) 

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])

# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')
